# BigData Capstone

## Group: U_Data

    Members:
    
    Aroob Almutairi 
    
    Abdulrahman Alosaimi
    
    Duaa Al saad 
    
    Fatimah Alateeq
    
    Ghaliah mohammedhussein


## Data describtion
- Convert U.S. to US to avoid periods.
- Convert Unknown to "?"

Data Set Information:

Extraction was done by Barry Becker from the 1994 Census database. A set of reasonably clean records was extracted using the following conditions: ((AAGE>16) && (AGI>100) && (AFNLWGT>1)&& (HRSWK>0))

Prediction task is to determine whether a person makes over 50K a year.

- age: continuous.
- workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.
- fnlwgt(final weight): The number of people the census takers believe that observation represents, continuous.
- education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.
- education-num: continuous.
- marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
- occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.
- relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.
- race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.
- sex: Female, Male.
- capital-gain: continuous.
- capital-loss: continuous.
- hours-per-week: continuous.
- native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.
- >50K, <=50K

- references:
- https://archive.ics.uci.edu/ml/datasets/adult

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
#lets import our usual suspects
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

#pyspark related imports
import pyspark
from pyspark.sql.functions import col, isnan, when, count
from pyspark.ml.feature import Imputer, VectorAssembler, StringIndexer
from pyspark.sql.functions import *
from pyspark.sql import functions as f
from pyspark.ml.regression import RandomForestRegressor, DecisionTreeRegressor, GBTRegressor, LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier, LogisticRegression, GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, CrossValidatorModel
from pyspark.ml import Pipeline

In [3]:
spark = pyspark.sql.SparkSession.builder.config("spark.executor.memory", "8g").config("spark.driver.memory", "8g").getOrCreate()

## Preprocessing

1. Replacing the '?' in all the columns with Not-specified
2. Make the values for education consistant
3. Change the values for salary to be 0 if <=50k, and 1 if >50k
4. Summing the values for the fnlwgt column, and getting the percentage of fnlwgt
5. The diffrence of  capital gain and loss
6. Finding if someone is workholic which is 1, or no which is 0  

## PySpark

In [4]:
# Load the dataset.
dfs = spark.read.csv('adult1m.csv',inferSchema=True,header=False)

In [5]:
# Rename columns
columns_names=['age','workclass','fnlwgt','education','education_num','marital_status','occupation','relationship','race','gender','capital_gain','capital_loss','hours_per_week','native_country','salary']
i=0
for colm in columns_names:
    dfs=dfs.withColumnRenamed('_c{}'.format(i),colm)
    i=i+1

In [6]:
# The data Shape
print(dfs.count(), len(dfs.columns))
dfs.printSchema()

1000000 15
root
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: integer (nullable = true)
 |-- education: string (nullable = true)
 |-- education_num: integer (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- capital_gain: integer (nullable = true)
 |-- capital_loss: integer (nullable = true)
 |-- hours_per_week: integer (nullable = true)
 |-- native_country: string (nullable = true)
 |-- salary: string (nullable = true)



In [7]:
dfs.show(5)

+---+----------------+------+---------+-------------+------------------+-----------------+-------------+-----+------+------------+------------+--------------+--------------+------+
|age|       workclass|fnlwgt|education|education_num|    marital_status|       occupation| relationship| race|gender|capital_gain|capital_loss|hours_per_week|native_country|salary|
+---+----------------+------+---------+-------------+------------------+-----------------+-------------+-----+------+------------+------------+--------------+--------------+------+
| 39|       State-gov| 77516|Bachelors|           13|     Never-married|     Adm-clerical|Not-in-family|White|  Male|        2174|           0|            40| United-States| <=50K|
| 50|Self-emp-not-inc| 83311|Bachelors|           13|Married-civ-spouse|  Exec-managerial|      Husband|White|  Male|           0|           0|            13| United-States| <=50K|
| 38|         Private|215646|  HS-grad|            9|          Divorced|Handlers-cleaners|Not-i

In [8]:
# To EDA in SQL
dfs.registerTempTable('dfs')

C:\Users\rooob\anaconda3\lib\site-packages\pyspark\sql\dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [9]:
# Value counts each column
for i in dfs.columns:
    dfs_result = spark.sql('SELECT `{}`, count(`{}`) FROM dfs group by 1 order by 2 desc'.format(i,i)) #first we select the subset of data
    dfs_result.show(50)
# No weird values
# ? is the NaN values

+---+----------+
|age|count(age)|
+---+----------+
| 37|     14347|
| 35|     14257|
| 31|     14250|
| 30|     14241|
| 24|     14224|
| 29|     14215|
| 28|     14205|
| 25|     14188|
| 36|     14182|
| 41|     14172|
| 44|     14145|
| 45|     14103|
| 39|     14091|
| 43|     14080|
| 34|     14069|
| 22|     14052|
| 33|     14045|
| 20|     14042|
| 23|     14032|
| 38|     14031|
| 47|     14020|
| 26|     14018|
| 42|     13990|
| 32|     13971|
| 40|     13940|
| 53|     13917|
| 48|     13888|
| 27|     13883|
| 18|     13854|
| 21|     13848|
| 46|     13802|
| 49|     13793|
| 50|     13789|
| 51|     13782|
| 17|     13758|
| 19|     13756|
| 58|     13725|
| 57|     13719|
| 55|     13703|
| 54|     13671|
| 52|     13636|
| 66|     13619|
| 56|     13566|
| 65|     13564|
| 59|     13518|
| 70|     13511|
| 64|     13490|
| 67|     13460|
| 63|     13438|
| 78|     13411|
+---+----------+
only showing top 50 rows

+----------------+----------------+
|       workclass|co

+--------------+---------------------+
|hours_per_week|count(hours_per_week)|
+--------------+---------------------+
|            40|                25612|
|            50|                13029|
|            45|                12327|
|            60|                11804|
|            20|                11540|
|            35|                11485|
|            30|                11470|
|            25|                10936|
|            48|                10894|
|            15|                10841|
|            55|                10785|
|            38|                10709|
|            32|                10620|
|            36|                10617|
|            70|                10593|
|             8|                10583|
|            44|                10581|
|            16|                10550|
|            56|                10541|
|             2|                10499|
|            10|                10498|
|            90|                10495|
|            42|         

In [10]:
dfs.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in dfs.columns]).show()

+---+---------+------+---------+-------------+--------------+----------+------------+----+------+------------+------------+--------------+--------------+------+
|age|workclass|fnlwgt|education|education_num|marital_status|occupation|relationship|race|gender|capital_gain|capital_loss|hours_per_week|native_country|salary|
+---+---------+------+---------+-------------+--------------+----------+------------+----+------+------------+------------+--------------+--------------+------+
|  0|        0|     0|        0|            0|             0|         0|           0|   0|     0|           0|           0|             0|             0|     0|
+---+---------+------+---------+-------------+--------------+----------+------------+----+------+------------+------------+--------------+--------------+------+



### Replaceing ? for the workclass with Not-specified

In [11]:
dfs.select('workclass').distinct().collect()

[Row(workclass='Self-emp-not-inc'),
 Row(workclass='Local-gov'),
 Row(workclass='State-gov'),
 Row(workclass='Private'),
 Row(workclass='Without-pay'),
 Row(workclass='Federal-gov'),
 Row(workclass='Never-worked'),
 Row(workclass='?'),
 Row(workclass='Self-emp-inc')]

In [12]:
dfs=dfs.withColumn('workclass', regexp_replace('workclass', "\\?", 'Not-specified'))
dfs.select('workclass').distinct().collect()

[Row(workclass='Self-emp-not-inc'),
 Row(workclass='Local-gov'),
 Row(workclass='State-gov'),
 Row(workclass='Not-specified'),
 Row(workclass='Private'),
 Row(workclass='Without-pay'),
 Row(workclass='Federal-gov'),
 Row(workclass='Never-worked'),
 Row(workclass='Self-emp-inc')]

### Replaceing ? for the occupation with not specified

In [13]:
dfs.select('occupation').distinct().collect()

[Row(occupation='Sales'),
 Row(occupation='Exec-managerial'),
 Row(occupation='Prof-specialty'),
 Row(occupation='Handlers-cleaners'),
 Row(occupation='Farming-fishing'),
 Row(occupation='Craft-repair'),
 Row(occupation='Transport-moving'),
 Row(occupation='Priv-house-serv'),
 Row(occupation='Protective-serv'),
 Row(occupation='Other-service'),
 Row(occupation='Tech-support'),
 Row(occupation='Machine-op-inspct'),
 Row(occupation='Armed-Forces'),
 Row(occupation='?'),
 Row(occupation='Adm-clerical')]

In [14]:
dfs=dfs.withColumn('occupation', regexp_replace('occupation', "\\?", 'Not-specified'))
dfs.select('occupation').distinct().collect()

[Row(occupation='Sales'),
 Row(occupation='Exec-managerial'),
 Row(occupation='Prof-specialty'),
 Row(occupation='Handlers-cleaners'),
 Row(occupation='Farming-fishing'),
 Row(occupation='Not-specified'),
 Row(occupation='Craft-repair'),
 Row(occupation='Transport-moving'),
 Row(occupation='Priv-house-serv'),
 Row(occupation='Protective-serv'),
 Row(occupation='Other-service'),
 Row(occupation='Tech-support'),
 Row(occupation='Machine-op-inspct'),
 Row(occupation='Armed-Forces'),
 Row(occupation='Adm-clerical')]

### Replaceing ? for the native_country with not specified

In [15]:
dfs.select('native_country').distinct().collect()

[Row(native_country='Philippines'),
 Row(native_country='Germany'),
 Row(native_country='Cambodia'),
 Row(native_country='France'),
 Row(native_country='Greece'),
 Row(native_country='Taiwan'),
 Row(native_country='Ecuador'),
 Row(native_country='Nicaragua'),
 Row(native_country='Hong'),
 Row(native_country='Peru'),
 Row(native_country='India'),
 Row(native_country='China'),
 Row(native_country='Italy'),
 Row(native_country='Holand-Netherlands'),
 Row(native_country='Cuba'),
 Row(native_country='South'),
 Row(native_country='Iran'),
 Row(native_country='Ireland'),
 Row(native_country='Thailand'),
 Row(native_country='Laos'),
 Row(native_country='El-Salvador'),
 Row(native_country='Mexico'),
 Row(native_country='Guatemala'),
 Row(native_country='Honduras'),
 Row(native_country='Yugoslavia'),
 Row(native_country='Puerto-Rico'),
 Row(native_country='Jamaica'),
 Row(native_country='Canada'),
 Row(native_country='United-States'),
 Row(native_country='Dominican-Republic'),
 Row(native_countr

In [16]:
dfs=dfs.withColumn('native_country', regexp_replace('native_country', "\\?", 'Not-specified'))
dfs.select('native_country').distinct().collect()

[Row(native_country='Philippines'),
 Row(native_country='Germany'),
 Row(native_country='Cambodia'),
 Row(native_country='France'),
 Row(native_country='Greece'),
 Row(native_country='Taiwan'),
 Row(native_country='Ecuador'),
 Row(native_country='Nicaragua'),
 Row(native_country='Hong'),
 Row(native_country='Peru'),
 Row(native_country='India'),
 Row(native_country='China'),
 Row(native_country='Not-specified'),
 Row(native_country='Italy'),
 Row(native_country='Holand-Netherlands'),
 Row(native_country='Cuba'),
 Row(native_country='South'),
 Row(native_country='Iran'),
 Row(native_country='Ireland'),
 Row(native_country='Thailand'),
 Row(native_country='Laos'),
 Row(native_country='El-Salvador'),
 Row(native_country='Mexico'),
 Row(native_country='Guatemala'),
 Row(native_country='Honduras'),
 Row(native_country='Yugoslavia'),
 Row(native_country='Puerto-Rico'),
 Row(native_country='Jamaica'),
 Row(native_country='Canada'),
 Row(native_country='United-States'),
 Row(native_country='Do

### Make the values for education consistant

In [17]:
dfs.select('education').distinct().collect()

[Row(education='Masters'),
 Row(education='10th'),
 Row(education='5th-6th'),
 Row(education='Assoc-acdm'),
 Row(education='Assoc-voc'),
 Row(education='7th-8th'),
 Row(education='9th'),
 Row(education='HS-grad'),
 Row(education='Bachelors'),
 Row(education='11th'),
 Row(education='1st-4th'),
 Row(education='Preschool'),
 Row(education='12th'),
 Row(education='Doctorate'),
 Row(education='Some-college'),
 Row(education='Prof-school')]

In [18]:
dfs=dfs.withColumn('education', regexp_replace('education', "5th-6th", '6th'))
dfs=dfs.withColumn('education', regexp_replace('education', "7th-8th", '8th'))
dfs=dfs.withColumn('education', regexp_replace('education', "1st-4th", '4th'))

dfs.select('education').distinct().collect()

[Row(education='Masters'),
 Row(education='10th'),
 Row(education='6th'),
 Row(education='Assoc-acdm'),
 Row(education='Assoc-voc'),
 Row(education='9th'),
 Row(education='HS-grad'),
 Row(education='Bachelors'),
 Row(education='11th'),
 Row(education='Preschool'),
 Row(education='8th'),
 Row(education='12th'),
 Row(education='Doctorate'),
 Row(education='4th'),
 Row(education='Some-college'),
 Row(education='Prof-school')]

### Change the values for salary to be 0 if <=50k, and 1 if >50k

In [19]:
dfs.select('salary').distinct().collect()

[Row(salary='<=50K'), Row(salary='>50K')]

In [20]:
dfs=dfs.withColumn('salary', regexp_replace('salary', "<=50K", '50000'))
dfs=dfs.withColumn('salary', regexp_replace('salary', ">50K", '50001'))
dfs = dfs.withColumn("salary", dfs["salary"].cast('int'))

dfs.select('salary').distinct().collect()

[Row(salary=50000), Row(salary=50001)]

In [21]:
dfs = dfs.withColumn("salary", dfs["salary"].cast('string'))

dfs=dfs.withColumn('salary', regexp_replace('salary', "50000", '0'))
dfs=dfs.withColumn('salary', regexp_replace('salary', "50001", '1'))
dfs = dfs.withColumn("salary", dfs["salary"].cast('int'))

dfs.select('salary').distinct().collect()

[Row(salary=1), Row(salary=0)]

### Summing the values for the fnlwgt column, and getting the percentage of fnlwgt

In [22]:
sum_fnlwgt=dfs.select(sum('fnlwgt')).collect()[0][0]
sum_fnlwgt

196336661945

In [23]:
dfs=dfs.withColumn('fnlwgt_percentage',round((dfs.fnlwgt/sum_fnlwgt)*100,2))
dfs.show()

+---+----------------+------+------------+-------------+--------------------+-----------------+-------------+------------------+------+------------+------------+--------------+--------------+------+-----------------+
|age|       workclass|fnlwgt|   education|education_num|      marital_status|       occupation| relationship|              race|gender|capital_gain|capital_loss|hours_per_week|native_country|salary|fnlwgt_percentage|
+---+----------------+------+------------+-------------+--------------------+-----------------+-------------+------------------+------+------------+------------+--------------+--------------+------+-----------------+
| 39|       State-gov| 77516|   Bachelors|           13|       Never-married|     Adm-clerical|Not-in-family|             White|  Male|        2174|           0|            40| United-States|     0|              0.0|
| 50|Self-emp-not-inc| 83311|   Bachelors|           13|  Married-civ-spouse|  Exec-managerial|      Husband|             White|  Ma

### The diffrence of  capital gain and loss

In [24]:
dfs=dfs.withColumn('diffrence_capital_gain_loos',(dfs.capital_gain-dfs.capital_loss))
dfs.show()

+---+----------------+------+------------+-------------+--------------------+-----------------+-------------+------------------+------+------------+------------+--------------+--------------+------+-----------------+---------------------------+
|age|       workclass|fnlwgt|   education|education_num|      marital_status|       occupation| relationship|              race|gender|capital_gain|capital_loss|hours_per_week|native_country|salary|fnlwgt_percentage|diffrence_capital_gain_loos|
+---+----------------+------+------------+-------------+--------------------+-----------------+-------------+------------------+------+------------+------------+--------------+--------------+------+-----------------+---------------------------+
| 39|       State-gov| 77516|   Bachelors|           13|       Never-married|     Adm-clerical|Not-in-family|             White|  Male|        2174|           0|            40| United-States|     0|              0.0|                       2174|
| 50|Self-emp-not-in

In [25]:
dfs=dfs.withColumn('profit',when((dfs.diffrence_capital_gain_loos >0), lit(1)).when((dfs.diffrence_capital_gain_loos ==0), lit(0)).when((dfs.diffrence_capital_gain_loos <0), lit(-1)))
dfs.show()

+---+----------------+------+------------+-------------+--------------------+-----------------+-------------+------------------+------+------------+------------+--------------+--------------+------+-----------------+---------------------------+------+
|age|       workclass|fnlwgt|   education|education_num|      marital_status|       occupation| relationship|              race|gender|capital_gain|capital_loss|hours_per_week|native_country|salary|fnlwgt_percentage|diffrence_capital_gain_loos|profit|
+---+----------------+------+------------+-------------+--------------------+-----------------+-------------+------------------+------+------------+------------+--------------+--------------+------+-----------------+---------------------------+------+
| 39|       State-gov| 77516|   Bachelors|           13|       Never-married|     Adm-clerical|Not-in-family|             White|  Male|        2174|           0|            40| United-States|     0|              0.0|                       2174|

In [26]:
dfs.select('profit').distinct().collect()

[Row(profit=-1), Row(profit=1), Row(profit=0)]

In [27]:
dfs.filter(dfs.profit == -1).show(truncate=False)

+---+----------------+------+------------+-------------+------------------+----------------+-------------+------------------+------+------------+------------+--------------+--------------+------+-----------------+---------------------------+------+
|age|workclass       |fnlwgt|education   |education_num|marital_status    |occupation      |relationship |race              |gender|capital_gain|capital_loss|hours_per_week|native_country|salary|fnlwgt_percentage|diffrence_capital_gain_loos|profit|
+---+----------------+------+------------+-------------+------------------+----------------+-------------+------------------+------+------------+------------+--------------+--------------+------+-----------------+---------------------------+------+
|43 |Private         |117037|11th        |7            |Married-civ-spouse|Transport-moving|Husband      |White             |Male  |0           |2042        |40            |United-States |0     |0.0              |-2042                      |-1    |
|45 

### Finding if someone is workholic which is 1, or no which is 0

In [28]:
dfs=dfs.withColumn('workaholic',when((dfs.hours_per_week >=70), lit(1)).when((dfs.hours_per_week <70), lit(0)))
dfs.show()

+---+----------------+------+------------+-------------+--------------------+-----------------+-------------+------------------+------+------------+------------+--------------+--------------+------+-----------------+---------------------------+------+----------+
|age|       workclass|fnlwgt|   education|education_num|      marital_status|       occupation| relationship|              race|gender|capital_gain|capital_loss|hours_per_week|native_country|salary|fnlwgt_percentage|diffrence_capital_gain_loos|profit|workaholic|
+---+----------------+------+------------+-------------+--------------------+-----------------+-------------+------------------+------+------------+------------+--------------+--------------+------+-----------------+---------------------------+------+----------+
| 39|       State-gov| 77516|   Bachelors|           13|       Never-married|     Adm-clerical|Not-in-family|             White|  Male|        2174|           0|            40| United-States|     0|             

### Replacing the native_country with in united state or not

In [29]:
dfs.select('native_country').distinct().collect()

[Row(native_country='Philippines'),
 Row(native_country='Germany'),
 Row(native_country='Cambodia'),
 Row(native_country='France'),
 Row(native_country='Greece'),
 Row(native_country='Taiwan'),
 Row(native_country='Ecuador'),
 Row(native_country='Nicaragua'),
 Row(native_country='Hong'),
 Row(native_country='Peru'),
 Row(native_country='India'),
 Row(native_country='China'),
 Row(native_country='Not-specified'),
 Row(native_country='Italy'),
 Row(native_country='Holand-Netherlands'),
 Row(native_country='Cuba'),
 Row(native_country='South'),
 Row(native_country='Iran'),
 Row(native_country='Ireland'),
 Row(native_country='Thailand'),
 Row(native_country='Laos'),
 Row(native_country='El-Salvador'),
 Row(native_country='Mexico'),
 Row(native_country='Guatemala'),
 Row(native_country='Honduras'),
 Row(native_country='Yugoslavia'),
 Row(native_country='Puerto-Rico'),
 Row(native_country='Jamaica'),
 Row(native_country='Canada'),
 Row(native_country='United-States'),
 Row(native_country='Do

In [30]:
dfs=dfs.withColumn('native_country', regexp_replace('native_country', "Philippines", 'Asia'))
dfs=dfs.withColumn('native_country', regexp_replace('native_country', "Germany", 'Europe'))
dfs=dfs.withColumn('native_country', regexp_replace('native_country', "Cambodia", 'Asia'))
dfs=dfs.withColumn('native_country', regexp_replace('native_country', "France", 'Europe'))
dfs=dfs.withColumn('native_country', regexp_replace('native_country', "Greece", 'Europe'))
dfs=dfs.withColumn('native_country', regexp_replace('native_country', "Taiwan", 'Asia'))
dfs=dfs.withColumn('native_country', regexp_replace('native_country', "Ecuador", 'South_America'))
dfs=dfs.withColumn('native_country', regexp_replace('native_country', "Nicaragua", 'Central_America'))
dfs=dfs.withColumn('native_country', regexp_replace('native_country', "Hong", 'Asia'))
dfs=dfs.withColumn('native_country', regexp_replace('native_country', "Peru", 'South_America'))
dfs=dfs.withColumn('native_country', regexp_replace('native_country', "India", 'Asia'))
dfs=dfs.withColumn('native_country', regexp_replace('native_country', "China", 'Asia'))
dfs=dfs.withColumn('native_country', regexp_replace('native_country', "Italy", 'Europe'))
dfs=dfs.withColumn('native_country', regexp_replace('native_country', "Holand-Netherlands", 'Europe'))
dfs=dfs.withColumn('native_country', regexp_replace('native_country', "Cuba", 'Caribbean'))
dfs=dfs.withColumn('native_country', regexp_replace('native_country', "Iran", 'Middle_East'))
dfs=dfs.withColumn('native_country', regexp_replace('native_country', "Ireland", 'Europe'))
dfs=dfs.withColumn('native_country', regexp_replace('native_country', "Thailand", 'Asia'))
dfs=dfs.withColumn('native_country', regexp_replace('native_country', "Laos", 'Asia'))
dfs=dfs.withColumn('native_country', regexp_replace('native_country', "El-Salvador", 'Central_America'))
dfs=dfs.withColumn('native_country', regexp_replace('native_country', "Mexico", 'North_America'))
dfs=dfs.withColumn('native_country', regexp_replace('native_country', "Guatemala", 'Central_America'))
dfs=dfs.withColumn('native_country', regexp_replace('native_country', "Honduras", 'Central_America'))
dfs=dfs.withColumn('native_country', regexp_replace('native_country', "Yugoslavia", 'Europe'))
dfs=dfs.withColumn('native_country', regexp_replace('native_country', "Puerto-Rico", 'Caribbean'))
dfs=dfs.withColumn('native_country', regexp_replace('native_country', "Jamaica", 'Caribbean'))
dfs=dfs.withColumn('native_country', regexp_replace('native_country', "Canada", 'North_America'))
dfs=dfs.withColumn('native_country', regexp_replace('native_country', "United-States", 'North_America'))
dfs=dfs.withColumn('native_country', regexp_replace('native_country', "Dominican-Republic", 'Caribbean'))
dfs=dfs.withColumn('native_country', regexp_replace('native_country', "Outlying-US\(Guam-USVI-etc\)", 'Caribbean'))
dfs=dfs.withColumn('native_country', regexp_replace('native_country', "Japan", 'Asia'))
dfs=dfs.withColumn('native_country', regexp_replace('native_country', "England", 'United_kingdom'))
dfs=dfs.withColumn('native_country', regexp_replace('native_country', "Haiti", 'Caribbean'))
dfs=dfs.withColumn('native_country', regexp_replace('native_country', "Poland", 'Europe'))
dfs=dfs.withColumn('native_country', regexp_replace('native_country', "Portugal", 'Europe'))
dfs=dfs.withColumn('native_country', regexp_replace('native_country', "Columbia", 'South_America'))
dfs=dfs.withColumn('native_country', regexp_replace('native_country', "Scotland", 'United_kingdom'))
dfs=dfs.withColumn('native_country', regexp_replace('native_country', "Hungary", 'Europe'))
dfs=dfs.withColumn('native_country', regexp_replace('native_country', "Vietnam", 'Asia'))
dfs=dfs.withColumn('native_country', regexp_replace('native_country', "Trinadad&Tobago", 'Caribbean'))

In [31]:
dfs=dfs.where(dfs.native_country!='South')

In [32]:
dfs.select('native_country').distinct().collect()

[Row(native_country='Middle_East'),
 Row(native_country='Europe'),
 Row(native_country='Central_America'),
 Row(native_country='Not-specified'),
 Row(native_country='Caribbean'),
 Row(native_country='South_America'),
 Row(native_country='North_America'),
 Row(native_country='United_kingdom'),
 Row(native_country='Asia')]

In [33]:
dfs.printSchema()

root
 |-- age: integer (nullable = true)
 |-- workclass: string (nullable = true)
 |-- fnlwgt: integer (nullable = true)
 |-- education: string (nullable = true)
 |-- education_num: integer (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- occupation: string (nullable = true)
 |-- relationship: string (nullable = true)
 |-- race: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- capital_gain: integer (nullable = true)
 |-- capital_loss: integer (nullable = true)
 |-- hours_per_week: integer (nullable = true)
 |-- native_country: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- fnlwgt_percentage: double (nullable = true)
 |-- diffrence_capital_gain_loos: integer (nullable = true)
 |-- profit: integer (nullable = true)
 |-- workaholic: integer (nullable = true)



In [34]:
pandasDF = dfs.toPandas()
pandasDF

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,salary,fnlwgt_percentage,diffrence_capital_gain_loos,profit,workaholic
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,North_America,0,0.0,2174,1,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,North_America,0,0.0,0,0,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,North_America,0,0.0,0,0,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,North_America,0,0.0,0,0,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Caribbean,0,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
976867,42,Never-worked,169069,Assoc-voc,2,Divorced,Sales,Husband,Asian-Pac-Islander,Male,2202,1340,37,Europe,1,0.0,862,1,0
976868,29,Self-emp-not-inc,192443,8th,3,Married-AF-spouse,Sales,Unmarried,Other,Male,2829,2246,32,Caribbean,1,0.0,583,1,0
976869,54,Federal-gov,36011,Bachelors,3,Married-spouse-absent,Armed-Forces,Unmarried,Amer-Indian-Eskimo,Female,1173,1594,97,North_America,1,0.0,-421,-1,1
976870,42,State-gov,197530,9th,4,Married-civ-spouse,Tech-support,Husband,White,Male,4416,1504,48,Caribbean,0,0.0,2912,1,0


In [35]:
pandasDF.to_csv("Cleaned_Data.csv",index=False)

# THANK YOU :)